In [ ]:
cd framework

In [ ]:
load("../framework/instance_gen.sage")
import numpy as np
import random

In [ ]:
n = 80
m = n
q = 3329
D_s = build_centered_binomial_law(40)
D_e = D_s
genarate_LWE_instance(DBDD, n, q, m, D_e, D_s)
A, b, s, dbdd = initialize_from_LWE_instance(DBDD, n, q, m, D_e, D_s)
beta, delta = dbdd.estimate_attack()

In [ ]:
# 使用拒绝采样生成[-3*sigma,3*sigma]之间满足高斯分布的随机数
def sample_discrete_gaussian(sigma):
    while True:
        x = random.randint(int(-3 * sigma), int(3 * sigma)) 
        acceptance_prob = math.exp(-x**2 / (2 * sigma**2)) 
        if random.uniform(0, 1) < acceptance_prob: 
            return x
        
def generate_se_eta_sigma_approx_hint(m, n, q, sigma, k):
  V = []
  L = []

  for i in range(k):
    D_v = build_centered_binomial_law(3)
    values, probabilities = zip(*D_v.items())
    v = np.array(np.random.choice(values, size=m+n, p=probabilities))
    noisy = sample_discrete_gaussian(sigma)
    l = dbdd.leak(v) + noisy
    V.append(v)
    L.append(l)
  return V,L

In [ ]:
num_hint = 60
sigma = 2
V, L = generate_se_eta_sigma_approx_hint(m, n, q, sigma, num_hint)
## 改成从文件中读取的函数

In [ ]:
output_dir = "/root/ShaoMingYao/Lattice_Reduction/DDGR20/leaky-LWE-Estimator-master/framework/Hints/Approx Hints/Kyber128/"

# 存储私钥es.txt
es = dbdd.u.submatrix(0, 0, 1, dbdd.u.ncols() - 1)
es_str = " ".join(map(str, es.list()))

es_file_path = os.path.join(output_dir, "es.txt")
with open(es_file_path, "w") as f:
    _ = f.write(es_str) # _ = 是为了显式处理返回值，否则f.write(es_str) 方法的返回值是写入文件的字符数
print(f"Secret es has been saved to {es_file_path}")

In [ ]:
# 存储系数v.txt

v_file_path = os.path.join(output_dir, "v.txt")
with open(v_file_path, "w") as vf:
    for row in V:
        _ = vf.write(" ".join(map(str, row)) + "\n")
print(f"Matrix V has been saved to {v_file_path}")

In [ ]:
# 存储值l.txt

l_file_path = os.path.join(output_dir, "l.txt")
with open(l_file_path, "w") as lf:
    for value in L:
        _ = lf.write(str(value) + "\n")
print(f"Vector L has been saved to {l_file_path}")

In [ ]:
e's'=es+randint(0,1)
b=As+e 
b-b'=A(s-s')+(e-e') 构造新的LWE问题
<v,s>=l
<v,s-s'>=l-<v,s'>  构造新的Hint